In [ ]:
import os
import cv2 as cv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from lib import app
from lib.utils import load_points, save_points, load_json, save_json, load_dlc_points_as_df, load_scene
from lib.plotting import plot_extrinsics
from lib.calib import triangulate_points

plt.style.use(os.path.join("..", "configs", "mplstyle.yaml"))

%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
int_path = "/Users/zico/Downloads/test2/calb_ext/manual_points/598"
ext_cam1_path = "/Users/zico/Downloads/test2/calb_ext/manual_points/601"
ext_cam2_path = "/Users/zico/Downloads/test2/calb_ext/manual_points/602"

In [ ]:
def convert_points(json_paths: list, out_fname: str):
    data = load_json(json_paths[0])
    output = {
        "board_shape": [9, 1],
        "board_square_len": 0,
        "camera_resolution": [data["imageWidth"], data["imageHeight"]],
        "points": {}
    }
    for fname in json_paths:
        data = load_json(fname)
        # assert len(data["shapes"]) == 9, f"File {fname} did not produce enough points"
        output["points"][data["imagePath"]] = []
        for i, p in enumerate(data["shapes"]):
            # assert f"p{i+1}" == p["label"]
            output["points"][data["imagePath"]].append(p["points"][0])

    save_json(out_fname, output)

In [ ]:
# Convert labelme points to acinoset points.
# Convert intrinsic points.
json_paths = sorted(glob(os.path.join(int_path, '*.json')))
convert_points(json_paths, os.path.join(os.path.dirname(int_path), "points3.json"))

# Convert extrinsic points.
json_paths = sorted(glob(os.path.join(ext_cam1_path, '*.json')))
convert_points(json_paths, os.path.join(os.path.dirname(ext_cam1_path), "points1.json"))

json_paths = sorted(glob(os.path.join(ext_cam2_path, '*.json')))
convert_points(json_paths, os.path.join(os.path.dirname(ext_cam2_path), "points2.json"))

In [ ]:
# Gather DLC labels of tail and spine and obtain pairwise points between the cameras to obtain an estimate of the extrinsics.
def get_pairwise_points_from_df(points_2d_df, camera_pairs):
    df_list = []
    # get pairwise estimates
    for cam_a, cam_b in camera_pairs:
        d0 = points_2d_df[points_2d_df['camera'] == cam_a]
        d1 = points_2d_df[points_2d_df['camera'] == cam_b]
        intersection_df = d0.merge(d1, how='inner', on=['frame', 'marker'], suffixes=('_a', '_b'))
        if intersection_df.shape[0] > 0:
            print(f'Found {intersection_df.shape[0]} pairwise points between camera {cam_a} and {cam_b}')
            df_list.append(intersection_df)
        else:
            print(f'No pairwise points between camera {cam_a} and {cam_b}')

    return df_list


def get_points_file(n_pts, fname):
    try:
        output = load_json(fname)
    except FileNotFoundError:
        output = {
            "board_shape": [n_pts, 1],
            "board_square_len": 0,
            "camera_resolution": [0, 0],
            "points": {}
        }

    return output


def generate_pairwise_points(markers, points_df, cam_a_idx, cam_b_idx, out_dir):
    output_a = get_points_file(len(markers), os.path.join(out_dir, f"points{cam_a_idx+1}.json"))
    output_b = get_points_file(len(markers), os.path.join(out_dir, f"points{cam_b_idx+1}.json"))
    frames = points_df["frame"].unique()
    for fno in frames:
        temp_a = []
        temp_b = []
        for m in markers:
            pts_a = points_df.query(f"frame == {fno} and marker=='{m}'")[["x_a", "y_a"]].values
            pts_b = points_df.query(f"frame == {fno} and marker=='{m}'")[["x_b", "y_b"]].values
            temp_a.append(pts_a.squeeze().tolist() if len(pts_a) > 0 else [np.nan, np.nan])
            temp_b.append(pts_b.squeeze().tolist() if len(pts_b) > 0 else [np.nan, np.nan])
        if len(np.asarray(temp_a, object).shape) == 2 and len(np.asarray(temp_b, object).shape) == 2:
            # np.any(~np.isnan(temp_a)) and np.any(~np.isnan(temp_b))
            output_a["points"][f"frame{fno}.png"] = temp_a
            output_b["points"][f"frame{fno}.png"] = temp_b

    save_json(os.path.join(out_dir, f"points{cam_a_idx+1}.json"), output_a)
    save_json(os.path.join(out_dir, f"points{cam_b_idx+1}.json"), output_b)

def plot_points_2d(points_fpath, cam_idx = 0):
    points, _, _, _, _ = load_points(points_fpath)

    plt.figure()
    plt.scatter(points[:, 0, 0], points[:, 0, 1], label="tail2")
    plt.scatter(points[:, 1, 0], points[:, 1, 1], label="r_back_paw")
    plt.scatter(points[:, 2, 0], points[:, 2, 1], label="l_front_ankle")
    plt.xlim((0, 800 if cam_idx < 2 else 1500))
    plt.ylim((0, 600 if cam_idx < 2 else 500))
    plt.legend()
    plt.show(block=False)


def plot_points_3d(obj_pts, title):
    fig = plt.figure()
    gs = fig.add_gridspec(2, 3)
    # fig, ax = plt.subplots(2, 3)
    ax1 = fig.add_subplot(gs[0, 0])
    ax2 = fig.add_subplot(gs[0, 1])
    ax3 = fig.add_subplot(gs[0, 2])
    # ax[0, 0].plot(obj_pts[:, 0])
    ax1.plot(obj_pts[:, 0])
    ax1.set_title("X")
    # ax[0, 1].plot(obj_pts[:, 1])
    ax2.plot(obj_pts[:, 1])
    ax2.set_title("Y")
    # ax[0, 2].plot(obj_pts[:, 2])
    ax3.plot(obj_pts[:, 1])
    ax3.set_title("Z")

    # ax = fig.add_subplot(2, 3, (4, 5, 6), projection='3d')
    ax4 = fig.add_subplot(gs[1, :], projection='3d')
    ax4.scatter(obj_pts[:, 0], obj_pts[:, 1], obj_pts[:, 2])
    ax4.set_xlabel("X")
    ax4.set_ylabel("Y")
    ax4.set_zlabel("Z")

    plt.title(title)
    plt.show(block=False)


In [ ]:
# Load DLC points.
dlc_dir = "/Users/zico/OneDrive - University of Cape Town/msc/data/cheetah_videos/kinetic_tests/2009_06_18/trail02/run/dlc"
points_dir = "/Users/zico/Downloads/test2/calb_ext"
n_cams = 3
dlc_points_fpaths = sorted(glob(os.path.join(dlc_dir, '*.h5')))
points_2d_df = load_dlc_points_as_df(dlc_points_fpaths, verbose=False)
filtered_points_2d_df = points_2d_df[points_2d_df['likelihood'] > 0.5]  # ignore points with low likelihood
filtered_points_2d_df = filtered_points_2d_df[['camera', 'frame', 'marker', 'x', 'y']]
# Add synchronisation offset to the third camera to sync it with cam1 and cam2.
filtered_points_2d_df.loc[filtered_points_2d_df['camera'] == 2, "frame"] += 259
camera_pairs = [[i % n_cams, (i + 1) % n_cams] for i in range(n_cams)]
df_list = get_pairwise_points_from_df(filtered_points_2d_df, camera_pairs)
markers = ["tail2", "r_back_paw", "l_front_ankle"]
for i, (cam_a, cam_b) in enumerate(camera_pairs):
    points_df = df_list[i]
    generate_pairwise_points(markers, points_df, cam_a, cam_b, points_dir)

In [ ]:
# Plot points to visually inspect the 2D points used in the optimisation.
plot_points_2d(os.path.join("/Users/zico/Downloads/test2/calb_ext/points", "points3.json"), 2)

In [ ]:
# Intrinsic calibration.
int_path = "/Users/zico/OneDrive - University of Cape Town/msc/data/cheetah_videos/kinetic_tests/intrinsic_calib/trail02/cam2"
K, D, R, t, used_points = app.calibrate_arabia_intrinsics(os.path.join(int_path, "points.json"), os.path.join(int_path, "camera.json"))

In [ ]:
# Extrinsic calibration.
points_fpaths = sorted(glob(os.path.join("/Users/zico/Downloads/test2/calb_ext/near_side/points", "points[1-4].json")))
scene_fpath = os.path.join("/Users/zico/Downloads/test2/calb_ext/near_side", f"{len(points_fpaths)}_cam_scene.json")
app.calibrate_arabia_extrinsics(
    scene_fpath, points_fpaths[0], points_fpaths[1],
    "/Users/zico/OneDrive - University of Cape Town/msc/data/cheetah_videos/kinetic_tests/intrinsic_calib/trail02/cam1/camera.json",
    "/Users/zico/OneDrive - University of Cape Town/msc/data/cheetah_videos/kinetic_tests/intrinsic_calib/trail02/cam2/camera.json"
)

In [ ]:
# Perform SBA on calibration points to refine extrinsic calibration parameters.
scene_fpath = os.path.join("/Users/zico/Downloads/test2/calb_ext/points", "3_cam_scene_initial.json")
points_fpaths = sorted(glob(os.path.join("/Users/zico/Downloads/test2/calb_ext/points", "points[1-4].json")))
scene_sba_fpath = scene_fpath.replace('.json','_sba.json')

res, pts_3d, obj_pts = app.sba_extrinsic_params_standard(
    scene_fpath, points_fpaths, out_fpath=scene_sba_fpath,
)

%matplotlib
plt.plot(res['before'], label='Cost Before')
plt.plot(res['after'], label='Cost After')
plt.legend()
plt.show(block=False)

plot_points_3d(pts_3d, "Estimate Before")
plot_points_3d(obj_pts, "Estimate After")

In [ ]:
scene_fpath = os.path.join("/Users/zico/Downloads/test2/calb_ext/points", "3_cam_scene_initial_sba.json")
scene_sba_fpath = scene_fpath.replace('.json','_sba_final.json')
# Load Measurement Data (pixels, likelihood)
points_2d_df = load_dlc_points_as_df(dlc_points_fpaths[:-1], verbose=False)
points_2d_df = points_2d_df[points_2d_df['frame'].between(600, 1000)]
points_2d_df = points_2d_df[points_2d_df['likelihood']>0.7] # ignore points with low likelihood

pts_3d, res = app.sba_points_standard(
    scene_fpath, points_2d_df,
)

%matplotlib inline
plt.plot(res['before'], label='Cost Before')
plt.plot(res['after'], label='Cost After')
plt.legend()
plt.show(block=False)

plot_points_3d(pts_3d, "Estimation of points")

In [ ]:
def plot_scene(scene_fpath, points_dir):
    pts_2d, frames = [], []
    points_fpaths = sorted(glob(os.path.join(points_dir, 'points[1-9].json')))
    for fpath in points_fpaths:
        img_pts, img_names, *_ = load_points(fpath)
        pts_2d.append(img_pts)
        frames.append(img_names)

    plot_extrinsics(scene_fpath, pts_2d, frames, triangulate_points)

In [ ]:
scene_fpath = os.path.join("/Users/zico/Downloads/test2/calb_ext/points", "3_cam_scene_initial_sba_final.json")
plot_scene(scene_fpath, "/Users/zico/Downloads/test2/calb_ext/points")

In [ ]:
# Moving new points into the existing points.
points = load_json("/Users/zico/Downloads/test2/calb_ext/points3.json")
points_orig = load_json("/Users/zico/Downloads/test2/calb_ext/points/points3_orig.json")
_, frames, board_shape, board_square_len, cam_res = load_points("/Users/zico/Downloads/test2/calb_ext/points/points3_orig.json")
_, new_frames, *_ = load_points("/Users/zico/Downloads/test2/calb_ext/points3.json")
for frame in new_frames:
    if frame in frames:
        points_orig["points"][frame][0][0] = points["points"][frame][0][0]
save_json("/Users/zico/Downloads/test2/calb_ext/points3_.json", points_orig)